In [1]:
#importing the library
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from mtcnn import MTCNN
from PIL import Image
import os


In [2]:
feature_list = np.array(pickle.load(open('embedding.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))

In [3]:
model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')
detector = MTCNN()

In [4]:
def upload_img(img_path):
    
    # load img -> face detection
    sample_img = cv2.imread(img_path) # Set the sample image to test. 
    results = detector.detect_faces(sample_img)
    x,y,width,height = results[0]['box']
    face = sample_img[y:y+height,x:x+width]
    
    #show the sample img
    position=(10,30)
    img = cv2.imread(img_path)
    img=cv2.resize(img,(400,400))
    img=cv2.putText(img,"Sample image",position,cv2.FONT_HERSHEY_SIMPLEX, .7, (255,0,0), 3) 
    #cv2.imshow('output',img)
    #cv2.waitKey(0)
    
    # extract its features
    image = Image.fromarray(face)
    image = image.resize((224,224))
    face_array = np.asarray(image)
    face_array = face_array.astype('float32')
    expanded_img = np.expand_dims(face_array,axis=0)
    preprocessed_img = preprocess_input(expanded_img)
    result = model.predict(preprocessed_img).flatten()
    
    # calculate the similar score and sort it and grab the most similar img index.
    similarity = []
    for i in range(len(feature_list)):
        similarity.append(cosine_similarity(result.reshape(1,-1),feature_list[i].reshape(1,-1))[0][0])
        index_pos = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][0]
    
    # show the most PREDICTED img
    predicted_actor =str("Looks like: ")+ " ".join(filenames[index_pos].split('\\')[1].split('_'))
    temp_img = cv2.imread(filenames[index_pos])
    temp_img=cv2.resize(temp_img,(400,400))
    temp_img=cv2.putText(temp_img,predicted_actor,position,cv2.FONT_HERSHEY_SIMPLEX, .7, (255,0,0), 3)
    combine = cv2.hconcat([img,temp_img])
    cv2.imshow('output',combine)
    cv2.waitKey(0)

In [5]:
#set the test image ------> "sample/img name of the sample folder"

#call the function
upload_img("sample/srk1.jpg")

1/1 [==============================] - 1s 928ms/step
